In [1]:
import numpy as np
from scipy import constants as cst

import matplotlib.pyplot as plt
import tqdm

from centrex_tlf import states, hamiltonian, transitions, couplings, lindblad, utils
from centrex_tlf_julia_extension import lindblad_julia

In [2]:
plt.rcParams.update({"font.size": 15})

In [3]:
trans = [
    transitions.OpticalTransition(
        transitions.OpticalTransitionType.P, J_ground=2, F1=3 / 2, F=1
    ),
    transitions.MicrowaveTransition(
        J_ground=1,
        J_excited=2,
        electronic_ground=states.ElectronicState.X,
        electronic_excited=states.ElectronicState.X,
    ),
    transitions.MicrowaveTransition(
        J_ground=2,
        J_excited=3,
        electronic_ground=states.ElectronicState.X,
        electronic_excited=states.ElectronicState.X,
    ),
]

polarizations = [
    [couplings.polarization_Z],
    [couplings.polarization_X, couplings.polarization_Y],
    [couplings.polarization_X, couplings.polarization_Y],
]

transition_selectors = couplings.generate_transition_selectors(
    trans,
    polarizations,
)

In [4]:
v_to_Γ = utils.detuning.velocity_to_detuning(1)
v_to_Γ_μ = 0

odepars = lindblad_julia.odeParameters(
    Ω0="Ωl0 * phase_modulation(t, β, ωphase)",
    Ωl0=1 * hamiltonian.Γ,  # Rabi frequency of the laser [rad/s]
    δ0=f"vx*{v_to_Γ}",  # detuning of the laser [rad/s]
    # laser phase modulation
    ωphase=hamiltonian.Γ,  # laser phase modulation frequency [rad/s]
    β=0,  # laser phase modulation depth [rad]
    PZ0=1,
    # J=1 -> J=2 microwaves
    Ω1=1 * hamiltonian.Γ,
    δ1=f"vx*{v_to_Γ_μ}",
    P1="sin(ω1*t + φ1)",
    PX1="P1>0",
    PY1="P1<=0",
    ω1=2 * np.pi * 1.56e6,  # polarization switching frequency [rad/s]
    φ1=0.0,  # polarization switching phase [rad]
    # J=2 -> J=3 microwaves
    Ω2=1 * hamiltonian.Γ,
    δ2=f"vx*{v_to_Γ_μ}",
    P2="sin(ω2*t + φ2)",
    PX2="P2>0",
    PY2="P2<=0",
    ω2=2 * np.pi * 1.56e6,  # polarization switching frequency [rad/s]
    φ2=0.0,  # polarization switching phase [rad]
    # molecules
    z0=0,  # molecule start z position [m]
    vz=184,  # longitudinal molecular velocity [m/s]
    vx=0,
)

In [5]:
obe_system = lindblad.setup_OBE_system_transitions(
    trans, transition_selectors, verbose=True, qn_compact=False, decay_channels=None
)

INFO:centrex_tlf.lindblad.utils_setup:generate_OBE_system: 1/5 -> Generating the reduced Hamiltonian
INFO:centrex_tlf.lindblad.utils_setup:generate_OBE_system: 2/5 -> Generating the couplings corresponding to the transitions
INFO:centrex_tlf.lindblad.utils_setup:generate_OBE_system: 3/5 -> Generating the symbolic Hamiltonian
INFO:centrex_tlf.lindblad.utils_setup:generate_OBE_system: 4/5 -> Generating the collapse matrices


INFO:centrex_tlf.lindblad.utils_setup:generate_OBE_system: 5/5 -> Transforming the Hamiltonian and collapse matrices into a symbolic system of equations


CPU times: total: 3min 1s
Wall time: 3min 9s


In [6]:
obe_system = lindblad_julia.setup_OBE_system_julia(
    obe_system, transition_selectors, odepars
)

In [7]:
couplings.generate_br_dataframe(obe_system.ground, obe_system.excited, group_ground="J")

,"|B, J = 1, F₁ = 3/2, F = 1>"
states,
"|X, J = 0>",0.484129
"|X, J = 2>",0.515871


In [8]:
couplings.generate_br_dataframe(
    obe_system.ground, obe_system.excited, group_ground="mF"
)

,"|B, J = 1, F₁ = 3/2, F = 1>"
states,
"|X, J = 0, F₁ = 1/2, F = 0>",0.336756
"|X, J = 0, F₁ = 1/2, F = 1>",0.147373
"|X, J = 2, F₁ = 3/2, F = 1>",0.100763
"|X, J = 2, F₁ = 3/2, F = 2>",0.026135
"|X, J = 2, F₁ = 5/2, F = 2>",0.388973


In [47]:
from typing import Union, Sequence
import numpy.typing as npt
from centrex_tlf.utils.population import thermal_population, J_levels


def generate_thermal_population_states(
    temperature: float,
    QN: Sequence[states.State],
) -> npt.NDArray[np.complex_]:
    levels = len(QN)
    ρ = np.zeros([levels, levels], dtype=complex)

    assert isinstance(QN[0], states.State), "no State objects supplies"

    j_levels = np.unique([qn.largest.J for qn in QN])

    # get the relative thermal population fractions of the ground state
    population = dict(
        [(j, p) for j, p in zip(j_levels, thermal_population(j_levels, temperature))]
    )

    # get quantum numbers of the ground state
    quantum_numbers = [
        (qn.largest.J, qn.largest.F1, qn.largest.F, qn.largest.mF)
        for qn in QN
        if qn.largest.electronic_state == states.ElectronicState.X
    ]

    assert len(np.unique(quantum_numbers, axis=0)) == len(
        quantum_numbers
    ), "duplicate quantum numbers"

    for idx, qn in enumerate(QN):
        if qn.largest.F is None:
            ρ[idx, idx] = population[qn.largest.J]
        else:
            ρ[idx,idx] = population[qn.largest.J] / J_levels(qn.largest.J)

    return ρ

In [52]:
ρ = generate_thermal_population_states(
    temperature = 6.3,
    QN = obe_system.ground,
)
ρ /= np.trace(ρ)

In [57]:
power = 85e-3 # * 11 / 2  # 11 passes assume zero intensity left after 11 passes
intensity = power / (25.4e-3 * 25.4e-3)
rabi = utils.rabi.intensity_to_rabi(
    intensity, obe_system.couplings[0].main_coupling, D=2.6675506e-30
).real

rabi_microwave = utils.rabi.power_to_rabi_gaussian_beam_microwave(
    power = 200e-3, coupling = obe_system.couplings[1].main_coupling, sigma_x = 25.4e-3, sigma_y=25.4e-3
).real

In [58]:
odepars.Ωl0 = rabi

problem = lindblad_julia.OBEProblem(odepars, ρ, tspan=(0, 25.4e-3 * 1.5 / odepars.vz))
config = lindblad_julia.OBEProblemConfig(saveat=1e-6)

In [59]:
results = lindblad_julia.do_simulation_single(problem, config=config)

: 

In [ ]:
qn_select_excited = states.QuantumSelector(electronic=states.ElectronicState.B)
indices_excited = qn_select_excited.get_indices(obe_system.QN)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(results.t * odepars.vz, results.y.T, lw=2)
ax.set_xlabel("distance [m]")
ax.set_ylabel("population")
# ax.legend(fontsize = 14)
ax.grid(True)

nphotons = np.trapz(results.y[indices_excited], x=results.t).sum() * hamiltonian.Γ
print(f"{nphotons:.2f} photons")
print(f"gain = {results.y[:4,-1].sum()/results.y[:4,0].sum()}")
